In [1]:
from bs4 import BeautifulSoup as soup
from urllib.request import urlopen as ureq
from selenium import webdriver
import time
import re

In [2]:
url = "https://www.ulaval.ca/etudes/programmes/baccalaureat-en-genie-mecanique#section-structure"

In [3]:
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--ignore-certificate-errors')
chrome_options.add_argument('--incognito')
#chrome_options.add_argument('--headless')

driver = webdriver.Chrome("C:\\Users\\jerry\\Downloads\\chromedriver", options=chrome_options)

In [4]:
driver.get(url)

# 1. Collect course link texts for webdriver to click on

In [6]:
driver.find_element_by_id("genie-mecanique-cours-obligatoires").click()

In [7]:
page_soup = soup(driver.page_source, 'lxml')

In [8]:
containers = page_soup.find("section", {"id": "section-structure"}).findAll("div", {"class": "fe-bloc-section"})[0].findAll("div", {"class": "toggle-section"})[0].find("ul", {"class": "fe--liste-cours"}).findAll("a")
len(containers)

33

In [9]:
link_texts = [container.text.strip() for container in containers]
link_texts

['GLO-1901\nIntroduction à la programmation avec Python',
 'GMC-1001\nStatique des corps rigides',
 'GMC-1024\nModélisation 3D',
 "GML-1001\nMatériaux de l'ingénieur",
 "MAT-1900\nMathématiques de l'ingénieur I",
 'GMC-1003\nIntroduction à la mécanique des fluides',
 'GMC-2001\nRésistance des matériaux',
 "MAT-1910\nMathématiques de l'ingénieur II",
 "MAT-2910\nAnalyse numérique pour l'ingénieur",
 'GMC-1002\nIntroduction à la thermodynamique',
 'GMC-2002\nRésistance des composantes de machines et des assemblages',
 'GMC-2003\nDynamique des corps rigides',
 'GMC-2005\nDynamique des fluides appliquée',
 'GMC-2006\nÉquations aux dérivées partielles en génie mécanique',
 'GMC-2024\nIngénierie et conception I',
 'ECN-2901\nAnalyse économique en ingénierie',
 'GMC-2008\nMachines thermiques',
 'GMC-2025\nIngénierie et conception II',
 'GMC-3000\nDynamique des vibrations',
 'GMC-3002\nDynamique de la commande appliquée',
 'GMC-3005\nTransferts thermiques',
 'GMC-3009\nGestion de projets en in

# 2. Test run - try to scrape the first course

In [10]:
driver.find_element_by_link_text(link_texts[0]).click()

ElementClickInterceptedException: Message: element click intercepted: Element is not clickable at point (547, -1628)
  (Session info: chrome=92.0.4515.107)


element not completely in view, so couldn't be clicked

In [12]:
from selenium.webdriver.common.keys import Keys
link = driver.find_element_by_link_text(link_texts[0])
link.send_keys(Keys.UP)
link.click()

In [13]:
page_soup = soup(driver.page_source, 'lxml')

In [14]:
course_title_container = page_soup.find("h1", {"class": "fe--titre"})

In [16]:
course_code = course_title_container.find("span", {"class": "fe--titre-type"}).text.strip()
course_code

'GLO-1901'

In [17]:
course_name = course_title_container.find("span", {"class": "fe--titre-nom"}).text.strip()
course_name

'Introduction à la programmation avec Python'

In [19]:
course_desc = page_soup.find("p", {"class": "fe--intro"}).text.strip()
course_desc

'Ce cours introduit les principaux paradigmes de programmation servant à la résolution de problèmes en informatique. Il couvre notamment les programmations procédurale, fonctionnelle, orientée objet et événementielle. Pour illustrer ces paradigmes, il utilise le langage Python.'

In [20]:
driver.back()

# 3. Test run successful! Implement automation script to scrape all courses

In [24]:
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import ElementClickInterceptedException

course_codes = []
course_names = []
course_descs = []
counter = 0
    
for link_text in link_texts:
    #go to course page
    try:
        link = driver.find_element_by_link_text(link_text)
    except NoSuchElementException:
        print("no link for {}".format(link_text))
        continue

    time.sleep(2)
    try:
        link.click()
    except ElementClickInterceptedException:
        link.send_keys(Keys.DOWN)
        link.click()
        
    time.sleep(3)
    
    #scrape course info
    page_soup = soup(driver.page_source, 'lxml')
    course_title_container = page_soup.find("h1", {"class": "fe--titre"})
    course_code = course_title_container.find("span", {"class": "fe--titre-type"}).text.strip()
    course_name = course_title_container.find("span", {"class": "fe--titre-nom"}).text.strip()
    course_desc = page_soup.find("p", {"class": "fe--intro"}).text.strip()
    
    course_codes.append(course_code)
    course_names.append(course_name)
    course_descs.append(course_desc)
    
    print("Scraped ", course_codes[-1], course_names[-1], "\n", course_descs[-1])
    counter += 1    
    print("{} courses scraped".format(counter))
    print()
    
    #go to course list page
    driver.back()
    time.sleep(3)
        
print("Finished scraping {} courses".format(counter))

Scraped  GLO-1901 Introduction à la programmation avec Python 
 Ce cours introduit les principaux paradigmes de programmation servant à la résolution de problèmes en informatique. Il couvre notamment les programmations procédurale, fonctionnelle, orientée objet et événementielle. Pour illustrer ces paradigmes, il utilise le langage Python.
1 courses scraped

Scraped  GMC-1001 Statique des corps rigides 
 Statique des corps solides. Notion de force. Systèmes de forces : lois fondamentales de l'équilibre dans le plan et dans l'espace. Méthodes vectorielles. Treillis, cadres et machines. Application aux mécanismes simples. Centre de masse. Poutres et câbles. Frottement de Coulomb et glissement. Travaux virtuels.
2 courses scraped

Scraped  GMC-1024 Modélisation 3D 
 Introduction au dessin à vues multiples et au dessin isométrique, lignes cachées, représentation des filets, interprétation des dessins à vues multiples, coupes et sections, vues auxiliaire, concept de modélisation paramétriqu

Scraped  GMC-2025 Ingénierie et conception II 
 Principes de modélisation CAO; conception des arbres et évaluation de la durée de vie en fatigue; les roulements et les principes de base d'utilisation et de montage des roulements; principes de modélisation physique (identification des aspects les plus importants d'un problème physique et création d'un modèle informatique ou théorique simplifié pour résoudre le problème); introduction aux calculs par éléments finis; introduction à la fabrication additive; principes de préparation et d'exécution d'une présentation orale, présentation orale; réalisation de projets pour consolider les notions d'ingénierie acquises; conférence d'un représentant technique provenant de l'industrie.
18 courses scraped

Scraped  GMC-3000 Dynamique des vibrations 
 Vibrations à un degré de liberté : libres, forcées, amorties. Techniques de modélisation. Linéarisation. Excitation par force constante, par balourd et par mouvement du support. Isolation des vibration

Scraped  GMC-3035 Ingénierie et conception VI 
 Intégration des connaissances dans la réalisation d'un projet de conception mécanique en appliquant les étapes de la méthodologie du design. Conception d'un système mécanique complexe, à partir d'une problématique industrielle définie par un client. Analyses de problèmes, calculs d'ingénierie, production de dessins techniques complets de machines, procédés ou systèmes destinés à des applications industrielles, utilitaires ou récréatives. Mise en pratique et consolidation des aptitudes au travail en équipe, rédaction de rapports techniques, préparation d'une présentation orale. Contenu du projet orienté pour développer les qualités Q4, Q5, Q6, Q7 et Q11 dans les cours Ingénierie et conception IV, V et VI.
32 courses scraped

Scraped  PHI-3900 Éthique et professionnalisme 
 Les professionnels d'aujourd'hui font face à des situations qui exigent des compétences d'ordre éthique et des savoirs qui débordent les connaissances techniques propres

# 4. Inspect and write to CSV

In [25]:
course_codes

['GLO-1901',
 'GMC-1001',
 'GMC-1024',
 'GML-1001',
 'MAT-1900',
 'GMC-1003',
 'GMC-2001',
 'MAT-1910',
 'MAT-2910',
 'GMC-1002',
 'GMC-2002',
 'GMC-2003',
 'GMC-2005',
 'GMC-2006',
 'GMC-2024',
 'ECN-2901',
 'GMC-2008',
 'GMC-2025',
 'GMC-3000',
 'GMC-3002',
 'GMC-3005',
 'GMC-3009',
 'GMC-3024',
 'STT-1900',
 'GMC-2007',
 'GMC-3006',
 'GMC-3023',
 'GMC-3025',
 'GMC-3013',
 'GMC-3034',
 'PHI-2910',
 'GMC-3035',
 'PHI-3900']

In [26]:
course_names

['Introduction à la programmation avec Python',
 'Statique des corps rigides',
 'Modélisation 3D',
 "Matériaux de l'ingénieur",
 "Mathématiques de l'ingénieur I",
 'Introduction à la mécanique des fluides',
 'Résistance des matériaux',
 "Mathématiques de l'ingénieur II",
 "Analyse numérique pour l'ingénieur",
 'Introduction à la thermodynamique',
 'Résistance des composantes de machines et des assemblages',
 'Dynamique des corps rigides',
 'Dynamique des fluides appliquée',
 'Équations aux dérivées partielles en génie mécanique',
 'Ingénierie et conception I',
 'Analyse économique en ingénierie',
 'Machines thermiques',
 'Ingénierie et conception II',
 'Dynamique des vibrations',
 'Dynamique de la commande appliquée',
 'Transferts thermiques',
 'Gestion de projets en ingénierie',
 'Ingénierie et conception III',
 'Méthodes statistiques pour ingénieurs',
 'Fabrication mécanique',
 'Introduction à la mesure et à la mécatronique',
 'Éléments de transmission de puissance',
 'Ingénierie et 

In [27]:
course_descs

['Ce cours introduit les principaux paradigmes de programmation servant à la résolution de problèmes en informatique. Il couvre notamment les programmations procédurale, fonctionnelle, orientée objet et événementielle. Pour illustrer ces paradigmes, il utilise le langage Python.',
 "Statique des corps solides. Notion de force. Systèmes de forces\xa0: lois fondamentales de l'équilibre dans le plan et dans l'espace. Méthodes vectorielles. Treillis, cadres et machines. Application aux mécanismes simples. Centre de masse. Poutres et câbles. Frottement de Coulomb et glissement. Travaux virtuels.",
 "Introduction au dessin à vues multiples et au dessin isométrique, lignes cachées, représentation des filets, interprétation des dessins à vues multiples, coupes et sections, vues auxiliaire, concept de modélisation paramétrique par extrusion, révolution et autres méthodes plus avancées avec logiciel SolidWorks, axes, plans et entités de référence, règles de cotation des dessins, mise en plan, de

In [32]:
len(list(dict.fromkeys(course_codes)))

33

In [33]:
import pandas as pd

df = pd.DataFrame({
    "Course Number": course_codes,
    "Course Name": course_names,
    "Course Description": course_descs
})

df

,Course Number,Course Name,Course Description
0,GLO-1901,Introduction à la programmation avec Python,Ce cours introduit les principaux paradigmes d...
1,GMC-1001,Statique des corps rigides,Statique des corps solides. Notion de force. S...
2,GMC-1024,Modélisation 3D,Introduction au dessin à vues multiples et au ...
3,GML-1001,Matériaux de l'ingénieur,Méthodes de caractérisation des matériaux. Coh...
4,MAT-1900,Mathématiques de l'ingénieur I,Calcul différentiel des fonctions de plusieurs...
5,GMC-1003,Introduction à la mécanique des fluides,Propriétés du fluide et concepts fondamentaux....
6,GMC-2001,Résistance des matériaux,Efforts internes dans un système mécanique. Ré...
7,MAT-1910,Mathématiques de l'ingénieur II,"Intégrales simples, calcul formel et numérique..."
8,MAT-2910,Analyse numérique pour l'ingénieur,Calcul numérique. Algèbre linéaire. Résolution...
9,GMC-1002,Introduction à la thermodynamique,Concepts de base et définitions. Propriétés de...


In [34]:
df.to_csv('Laval_GenieMecanique_Core_Courses.csv', index = False)

In [35]:
driver.quit()